**Inputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair

**Outputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair--where magnitude values are thresholds

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWiNfEsn0a3nRbirupQBz_nDk1F1bsNQJ5CTHU3Z-wo1-yXScLlpk4U

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
import hazard_info

In [1]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
START_YEAR = 2020
END_YEAR = 2099
#PROBTYPE = "MAPprobs"
PROBTYPE = "posterior_probs"

NameError: name 'ee' is not defined

In [6]:
for hazard in hazard_info.hazards[-1:]:
    hazardname = hazard['name']
    use_greaterthan = hazard['use_greaterthan']
    mapprobs = ee.ImageCollection('users/tedwongwri/dataportal/{0}/{1}'.format(PROBTYPE, hazardname))
    #for year in range(START_YEAR, END_YEAR + 1):
    for year in [1980, 2030, 2050, 2080]:
        # Goes through bands, each of which has the probs for one magnitude interval
        # Then, for each band, goes through all magnitude intervals and adds probs if
        # the inner (more raidly changing) magnitude interval is >= or <= (as apporpriate)
        # the outer magnitude band
        year_list = ee.List([])
        img = mapprobs.filterMetadata('year', 'equals', year).first()
        band_names = img.bandNames().getInfo()
        mag_parts = [i.split('_')[1] for i in band_names]
        highlows = [[float(j.replace('neg', '-')) for j in i.split('-')] for i in mag_parts]
        for magnitude in highlows:
            total_img = ee.Image(0)
            prob_img = ee.Image(0)
            for hl1 in highlows:
                prob_segment = img.select('magnitude_{0}-{1}'.format(str(int(hl1[0])).replace('-', 'neg'), str(int(hl1[1])).replace('-', 'neg')))
                total_img = total_img.add(prob_segment)
                prob_img = prob_img.add(prob_segment.multiply(((hl1[1] >= magnitude[1] and use_greaterthan) or (hl1[0] <= magnitude[0] and not use_greaterthan)) * 1))
            year_list = year_list.add(prob_img.divide(total_img).set('threshold', [magnitude[1], magnitude[0]][use_greaterthan * 1]))
        year_img = ee.ImageCollection(year_list).toBands().rename(['threshold_{0}'.format(str(int(i[[1, 0][use_greaterthan * 1]])).replace('-', 'neg')) for i in highlows]).set('year', year)
        task = ee.batch.Export.image.toAsset(**{
            'image': year_img,
            'description': 'threshold{0}_{1}_{2}'.format(PROBTYPE, hazardname, str(year)),
            'assetId': 'users/tedwongwri/dataportal/threshold{0}/{1}/{2}'.format(PROBTYPE, hazardname, str(year)),
            'region': WHOLE_GLOBE,
            'crs': 'EPSG:4326',
            #'dimensions': '1440x720'
            'dimensions': '43200x21600',# For Aqueduct
            'maxPixels': 1000000000 # For Aqueduct
        })
        task.start()